# MNIST 模型强化

## 步骤1：载入 MNIST 手写阿拉伯数字资料

In [1]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

# 载入 MNIST 手写阿拉伯数字资料
(x_train, y_train),(x_test, y_test) = mnist.load_data()


## 步骤2：资料清理，此步骤无需进行

## 步骤3：进行特征工程，将特征缩放成(0, 1)之间

# 特征缩放，使用常态化(Normalization)，公式 = (x - min) / (max - min)
# 颜色范围：0~255，所以，公式简化为 x / 255
# 注意，颜色0为白色，与RGB颜色不同，(0,0,0) 为黑色。
x_train_norm, x_test_norm = x_train / 255.0, x_test / 255.0

## 步骤2：改用 CNN 模型

In [2]:
# 建立模型
from tensorflow.keras import layers
import numpy as np

input_shape=(28, 28, 1)
# 增加一维在最后面
x_train_norm = np.expand_dims(x_train_norm, -1)
x_test_norm = np.expand_dims(x_test_norm, -1)

# CNN 模型
model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(10, activation="softmax"),
    ]
)

In [3]:
# 设定优化器(optimizer)、损失函数(loss)、效能衡量指标(metrics)的类别
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 模型训练
history = model.fit(x_train_norm, y_train, epochs=5, validation_split=0.2)

# 评分(Score Model)
score=model.evaluate(x_test_norm, y_test, verbose=0)

for i, x in enumerate(score):
    print(f'{model.metrics_names[i]}: {score[i]:.4f}')

Epoch 1/5
1500/1500 [==============================] - 23s 15ms/step - loss: 0.5261 - accuracy: 0.8317 - val_loss: 0.0731 - val_accuracy: 0.9776
Epoch 2/5
1500/1500 [==============================] - 21s 14ms/step - loss: 0.0954 - accuracy: 0.9714 - val_loss: 0.0535 - val_accuracy: 0.9852
Epoch 3/5
1500/1500 [==============================] - 21s 14ms/step - loss: 0.0728 - accuracy: 0.9775 - val_loss: 0.0449 - val_accuracy: 0.9869
Epoch 4/5
1500/1500 [==============================] - 21s 14ms/step - loss: 0.0574 - accuracy: 0.9821 - val_loss: 0.0393 - val_accuracy: 0.9888
Epoch 5/5
1500/1500 [==============================] - 21s 14ms/step - loss: 0.0542 - accuracy: 0.9828 - val_loss: 0.0380 - val_accuracy: 0.9895
loss: 0.0298
accuracy: 0.9902


In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                1

In [5]:
import math

def Conv_Width(W, F, P, S):
    return math.floor(((W - F + 2 * P) / S) + 1)

def Pool_Width(W, F, S):
    return math.floor((W - F) / S + 1)

Conv_Width(28, 3, 0, 1)

26

In [6]:
Pool_Width(26, 2, 2)

13